In [ ]:
# MailSense — AI Email Summarizer & Action Planner


## Demo: Run refined agent pipeline

This demo runs the refined MailSense pipeline (extractor → refined planner → tools) on our example emails.
It demonstrates the following course concepts:
- **Tool Use**: create_task, schedule_event, compose_reply are called as tools.
- **Memory**: recent emails and task index are recorded in `memory.json`.
- **Planning**: `planner_refined` chooses actions and handles conflicts & priorities.

In [5]:
# Notebook cell: run refined agent and display results
from pathlib import Path
import json
from src.agent_main_refined import run_batch

results = run_batch()
print("Agent produced reports for", len(results), "emails.")
# print summaries in a readable form
for r in results:
    print("="*60)
    print("Email subject:", r["email"]["subject"])
    print("Summary:", r["email"]["summary"])
    for p in r["plans"]:
        print(" - Recommendation:", p["recommendation"], "| confidence:", p["confidence"], "| reason:", p["reason"])
        print("   Tool result:", p.get("tool_result"))


Agent produced reports for 6 emails.
Email subject: Meeting request
Summary: Meeting request
 - Recommendation: compose_reply | confidence: 0.6 | reason: Conflict detected with existing scheduled item
   Tool result: {'status': 'ok', 'reply_text': 'Hi there, thanks — could you confirm: which of these slots works for you?'}
Email subject: Invoice #1234 - due Nov 30
Summary: Invoice #1234 - due Nov 30
 - Recommendation: create_task | confidence: 0.97 | reason: Invoice/amount detected: ['$150']
   Tool result: {'status': 'ok', 'task_id': 't-7b362c39', 'message': 'Task created'}
Email subject: Weekly team update
Summary: Weekly team update. Team — here are the updates for this week: we finished feature X, started Y, and plan Z next week.
 - Recommendation: ignore | confidence: 0.55 | reason: No clear action
   Tool result: None
Email subject: Quick follow-up
Summary: Quick follow-up. Can you send the draft by Monday? Thanks.
 - Recommendation: create_task | confidence: 0.8 | reason: Generi

### Persisted tasks

Show content of `tasks.json` (tasks created by the agent).


In [6]:
from pathlib import Path
import json
p = Path("../tasks.json")  # adjust path if notebook root differs; use correct relative path
if p.exists():
    print(p.read_text())
else:
    print("No tasks.json found yet.")


{
  "tasks": [
    {
      "task_id": "t-dbf1f1f9",
      "title": "Schedule: Meeting request",
      "description": "Hi Arshwin â€” are you free Friday 3â€“4pm for a haircut? If not, next week Tue afternoon works. â€” Priya",
      "created_at": "2025-11-20T14:05:09.395071",
      "due": "2026-03-20T16:00:00",
      "status": "pending",
      "source_email_id": "e-21cff4ea",
      "priority": "medium",
      "tags": [
        "schedule"
      ]
    },
    {
      "task_id": "t-f279a265",
      "title": "Invoice Action: Invoice #1234 - due Nov 30",
      "description": "Dear Arshwin, your invoice #1234 for $150 is due on Nov 30. Please confirm payment.",
      "created_at": "2025-11-20T14:05:09.631369",
      "due": "1234-11-20T00:00:00",
      "status": "pending",
      "source_email_id": "e-527944b5",
      "priority": "high",
      "tags": [
        "invoice"
      ]
    },
    {
      "task_id": "t-f7f6dfe8",
      "title": "Task: Quick follow-up",
      "description": "Can you sen

## Where each course concept is implemented

- **Tool Use (Day 2)**: implemented through functions in `src/tools.py` and invoked by planner/tool dispatcher in `src/agent_main_refined.py`. Tools: `create_task`, `schedule_event`, `compose_reply`.
- **Memory (Day 3)**: persistent memory stored in `memory.json` via `src/memory.py` (functions `add_recent_email`, `add_task_index`).
- **Planning (Day 1 & 5)**: `src/planner_refined.py` implements decision rules, conflict detection, and priority inference.


In [1]:
import sys
print(sys.executable)


c:\Users\91759\Desktop\mail-sense\.venv\Scripts\python.exe


In [2]:
import importlib, pkgutil

required = ["dateparser", "python_dateutil", "pandas"]
for pkg in required:
    found = pkgutil.find_loader(pkg) is not None
    print(f"{pkg}: {'OK' if found else 'MISSING'}")


dateparser: OK
python_dateutil: MISSING
pandas: OK


C:\Users\91759\AppData\Local\Temp\ipykernel_14020\3014824243.py:5: DeprecationWarning: 'pkgutil.find_loader' is deprecated and slated for removal in Python 3.14; use importlib.util.find_spec() instead
  found = pkgutil.find_loader(pkg) is not None


In [3]:
# Notebook: ensure project root is on sys.path so "import src..." works
import sys
from pathlib import Path

# If notebook lives in mail-sense/notebooks, parent is project root
PROJECT_ROOT = Path("..").resolve()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print("Project root added to sys.path:", PROJECT_ROOT)


Project root added to sys.path: C:\Users\91759\Desktop\mail-sense


In [4]:
# Run refined agent and display results
from pathlib import Path
import json
from src.agent_main_refined import run_batch

results = run_batch()
print("Agent produced reports for", len(results), "emails.\n")

for i, r in enumerate(results, start=1):
    print("=" * 80)
    print(f"Email #{i}")
    print(" Subject :", r["email"]["subject"])
    print(" Summary :", r["email"]["summary"])
    print(" Plans  :")
    for p in r["plans"]:
        rec = p.get("recommendation")
        conf = p.get("confidence")
        reason = p.get("reason")
        print(f"  - {rec} (conf={conf}) — {reason}")
        tool_res = p.get("tool_result")
        if tool_res:
            if isinstance(tool_res, dict):
                if tool_res.get("task_id"):
                    print(f"     -> Task created: {tool_res.get('task_id')}")
                elif tool_res.get("reply_text"):
                    print(f"     -> Reply text: {tool_res.get('reply_text')}")
                else:
                    print(f"     -> Tool result: {tool_res}")
            else:
                print(f"     -> Tool result: {tool_res}")
    print()
    
proj_root = Path("..").resolve()
tasks_path = proj_root / "tasks.json"
mem_path = proj_root / "memory.json"
print("Artifacts:")
print(" tasks.json exists:", tasks_path.exists(), "-", str(tasks_path))
print(" memory.json exists:", mem_path.exists(), "-", str(mem_path))

if tasks_path.exists():
    print("\n--- tasks.json content ---")
    print(tasks_path.read_text())


Agent produced reports for 6 emails.

Email #1
 Subject : Meeting request
 Summary : Meeting request
 Plans  :
  - compose_reply (conf=0.6) — Conflict detected with existing scheduled item
     -> Reply text: Hi there, thanks — could you confirm: which of these slots works for you?

Email #2
 Subject : Invoice #1234 - due Nov 30
 Summary : Invoice #1234 - due Nov 30
 Plans  :
  - create_task (conf=0.97) — Invoice/amount detected: ['$150']
     -> Task created: t-72debb8f

Email #3
 Subject : Weekly team update
 Summary : Weekly team update. Team — here are the updates for this week: we finished feature X, started Y, and plan Z next week.
 Plans  :
  - ignore (conf=0.55) — No clear action

Email #4
 Subject : Quick follow-up
 Summary : Quick follow-up. Can you send the draft by Monday? Thanks.
 Plans  :
  - create_task (conf=0.8) — Generic task detected
     -> Task created: t-51cb5722

Email #5
 Subject : Schedule a call
 Summary : Schedule a call. Hi, can we schedule a quick call next

## Developer Plan (where code lives)

- src/agent/agent_main.py  -> ADK agent entrypoint (instantiates agent, registers tools, handles input)
- src/tools.py            -> Contains implementations for summarize_email, extract_actions, create_task, schedule_event, compose_reply, log_action
- src/memory.py           -> Read/write memory.json and helper functions
- src/planner.py          -> Implements planner rules (see src/planner_rules.md)
- src/utils.py            -> Utility functions: date parsing, similarity checking
- data/examples_emails.json -> Example inputs used for demonstration
- tasks.json, calendar.json, memory.json -> Persistent stores produced by the tools


In [ ]:
**Short pitch:** MailSense ingests email text, summarizes it, extracts actions (tasks, deadlines, contacts), recommends a next action (create task / schedule / reply template), and can execute a mock task creation. Demonstrates Tool Use, Memory, and Planning.

In [ ]:
**Demo GIF:** Demo GIF will go here (demo/demo.gif)

In [ ]:
**Concepts demonstrated:** 
1. Tool Use (registering and calling tools)
2. Memory (persistent storage of past tasks)
3. Planning / Multi-step reasoning (intent → extract → decide → act)

In [ ]:
**Example run:** (will show demonstration of loading an email, summarizing, extracting actions, and creating a task)

In [ ]:
**Reproducibility & run instructions:** See docs/methodology.md and README.md for exact environment & commands.

In [ ]:
**Results & Evaluation:** Placeholder for evaluation table and scoring of the sample emails.

In [ ]:
**Methodology:** See docs/methodology.md